In [1]:
import numpy as np
import sys
import time
import pandas as pd

In [2]:
class Dataset(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y

In [3]:
class TreeNode(object):
    def __init__(self):
        self.is_leaf = False
        self.left_child = None
        self.right_child = None
        self.split_feature = None
        self.split_value = None
        self.child_weight = None

    def calc_gain(self, G_l, G_r, H_l, H_r, lambd, gamma):
        """Measure how good a tree is. Equation 7"""
        def calc_term(g, h):
            return np.square(g) / (h + lambd)
        gain = 0.5 * (calc_term(G_l, H_l) + \
                      calc_term(G_r, H_r) - \
                      calc_term(G_l + G_r, H_l + H_r)) - gamma
        # the bigger gamma, the more convative
        return gain
    
    def calc_child_weight(self, g, h, lambd):
        """Calc the optimal weight of leaf node. Equation 5"""
        return - np.sum(g) / (np.sum(h) + lambd)
    
    def build(self, instances, grad, hessian, eta, depth, param):
        """Algorithm 1"""
        if depth > param['max_depth']:
            # If the depth now is bigger than max depth, it is leaf node, and stop growing.
            self.is_leaf = True
            self.weight = self.calc_child_weight(grad, hessian, param['lambda']) * eta
            return
        G = np.sum(grad)
        H = np.sum(hessian)
        best_gain = 0.
        best_feature = None
        best_val = 0.
        best_left_instances = None
        best_right_instances = None
        for feature in range(instances.shape[1]):
            G_l, H_l = 0., 0.
            sorted_instances = instances[:, feature].argsort()
            for j in range(len(sorted_instances)):
                G_l += grad[sorted_instances[j]]
                H_l += hessian[sorted_instances[j]]
                G_r = G - G_l
                H_r = H - H_l
                current_gain = self.calc_gain(G_l, H_l, G_r, H_r,
                                              param['lambda'], param['gamma'])
                if current_gain > best_gain:
                    best_gain = current_gain
                    best_feature = feature
                    best_val = instances[sorted_instances[j]][feature]
                    best_left_instances = sorted_instances[:j+1]
                    best_right_instances = sorted_instances[j+1:]
        if best_gain < param['min_split_gain']:
            self.is_leaf = True
            self.weight = self.calc_child_weight(grad, hessian, param['lambda']) * eta
        else:
            self.split_feature = best_feature
            self.split_value = best_val
            self.left_child = TreeNode()
            self.left_child.build(instances[best_left_instances],
                                  grad[best_left_instances],
                                  hessian[best_left_instances],
                                  eta, depth+1, param)
            
            self.right_child = TreeNode()
            self.right_child.build(instances[best_right_instances],
                                  grad[best_right_instances],
                                  hessian[best_right_instances],
                                  eta, depth+1, param)
    
    def predict(self, x):
        if self.is_leaf:
            return self.weight
        else:
            if x[self.split_feature] <= self.split_value:
                return self.left_child.predict(x)
            else:
                return self.right_child.predict(x)

In [4]:
class Tree(object):
    """Tree ensemble"""
    def __init__(self):
        self.root = None
    
    def build(self, instances, grad, hessian, eta, param):
        self.root = TreeNode()
        current_depth = 0
        self.root.build(instances, grad, hessian, eta, current_depth, param)
        
    def predict(self, x):
        return self.root.predict(x)

In [5]:
class GBT(object):
    def __init__(self):
        self.params = {'gamma': 0.,
                       'lambda': 1.,
                       'min_split_gain': 0.1,
                       'max_depth': 5,
                       'learning_rate': 0.3}
        self.best_iteration = None
        
    def calc_training_data_scores(self, train_set, models):
        if len(models) == 0:
            return None
        X = train_set.X
        scores = np.zeros(len(X))
        for i in range(len(X)):
            scores[i] = self.predict(X[i], models=models)
        return scores
    
    def calc_l2_gradient(self, train_set, scores):
        labels = train_set.y
        hessian = np.full(len(labels), -2)
        if scores is None:
            grad = np.random.uniform(size=len(labels))
        else:
            grad = np.array([2 * (labels[i] - scores[i]) for i in range(len(labels))])
        return grad, hessian
    
    def calc_l2_loss(self, models, data_set):
        errors = []
        X, y = data_set.X, data_set.y
        for x, y in zip(X, y):
            errors.append(y - self.predict(x, models))
        return np.mean(np.square(errors))
    
    def build_learner(self, train_set, grad, hessian, eta):
        learner = Tree()
        learner.build(train_set.X, grad, hessian, eta, self.params)
        return learner
    
    def train(self, params, train_set, valid_set=None, num_boost_rounds=20,
              early_stopping_rounds=5, calc_grad=None, calc_loss=None):
        self.params.update(params)
        models = []
        eta = self.params['learning_rate']
        best_iteration = None
        best_val_loss = np.infty
        start = time.time()
        
        for cnt in range(num_boost_rounds):
            iter_start = time.time()
            score = self.calc_training_data_scores(train_set, models)
            if calc_grad is None:
                grad, hessian = self.calc_l2_gradient(train_set, score)
            else:
                grad, hessian = calc_grad(train_set, score)
            learner = self.build_learner(train_set, grad, hessian, eta)
            models.append(learner)
            if calc_loss is None:
                train_loss = self.calc_l2_loss(models, train_set)
            else:
                train_loss = calc_loss(models, train_set)
            if valid_set is not None:
                if calc_loss is None:
                    val_loss = self.calc_l2_loss(models, valid_set)
                else:
                    val_loss = calc_loss(models, valid_set)
            else:
                val_loss = None
            if val_loss is not None and val_loss <= best_val_loss:
                best_val_loss = val_loss
                best_iteration = cnt
                if cnt - best_iteration >= early_stopping_rounds:
                    print('Early stopping, best iteration is: %d' %(best_iteration))
                    break
        self.models = models
        self.best_iteration = best_iteration
        print('Train finished. Elapsed: %.2fs, Loss: %.2f' %(time.time() - start, train_loss))
        
    def predict(self, x, models=None, num_iter=None):
        if models is None:
            models = self.models
        return sum(m.predict(x) for m in models[:num_iter])

In [6]:
df_train = pd.read_csv('regression.train', header=None, sep='\t')
df_test = pd.read_csv('regression.test', header=None, sep='\t')

In [7]:
y_train = df_train[0].values
y_test = df_test[0].values
X_train = df_train.drop(0, axis=1).values
X_test = df_test.drop(0, axis=1).values

train_data = Dataset(X_train, y_train)
eval_data = Dataset(X_test, y_test)

In [8]:
params = {}

print('Start training...')
gbt = GBT()
gbt.train(params,
          train_data,
          valid_set=eval_data,
          early_stopping_rounds=5)

Start training...
Train finished. Elapsed: 326.10s, Loss: 0.23


In [9]:
y_pred = []
for x in X_test:
    y_pred.append(gbt.predict(x, num_iter=gbt.best_iteration))

In [10]:
np.mean(y_test)

0.544

In [11]:
np.mean(y_pred)

0.5259516455147291

In [12]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [13]:
mean_squared_error(y_test, y_pred) ** 0.5

0.4855631253167693

In [14]:
y_pred = pd.Series(y_pred).apply(lambda x: 1 if x >= 0.5 else 0)

In [15]:
print(classification_report(digits=4, y_pred=y_pred, y_true=y_test))

              precision    recall  f1-score   support

           0     0.6081    0.3947    0.4787       228
           1     0.6080    0.7868    0.6859       272

   micro avg     0.6080    0.6080    0.6080       500
   macro avg     0.6080    0.5908    0.5823       500
weighted avg     0.6080    0.6080    0.5914       500



In [16]:
print(confusion_matrix(y_pred=y_pred, y_true=y_test))

[[ 90 138]
 [ 58 214]]
